In [1]:
import pandas as pd
import pubchempy as pcp
import os
from vina import Vina

rec_lig_df = pd.read_csv("IUPHAR/interactions.csv", skiprows=[0])
rec_lig_df = rec_lig_df[['Target', 'Target ID', 'Ligand', 'Ligand ID']].drop_duplicates().dropna()
TRPV1_lig_df = rec_lig_df[rec_lig_df['Target ID']==507.0].drop_duplicates()
TRPV1_lig_df.head()

/tmp/ipykernel_3427336/4274587102.py:6: DtypeWarning: Columns (6,8,9,10,11,39) have mixed types. Specify dtype option on import or set low_memory=False.
  rec_lig_df = pd.read_csv("IUPHAR/interactions.csv", skiprows=[0])


,Target,Target ID,Ligand,Ligand ID
21609,TRPV1,507.0,anandamide,2364
21610,TRPV1,507.0,DkTx,4178
21611,TRPV1,507.0,RhTx,12544
21612,TRPV1,507.0,[<sup>3</sup>H]resiniferatoxin,4091
21613,TRPV1,507.0,zucapsaicin,11576


In [2]:
def contains_invalid_chars(smiles):
    invalid_chars = ['(', ')', '<', '>']
    return any(char in smiles for char in invalid_chars)

TRPV_ligands = rec_lig_df[rec_lig_df['Target ID']==507.0].drop_duplicates()
TRPV_ligands = TRPV_ligands[~TRPV_ligands['Ligand'].apply(contains_invalid_chars)]
TRPV_ligands = TRPV_ligands['Ligand'].tolist()
len(TRPV_ligands)

44

In [5]:
!ls IUPHAR/IUPHAR_dock/

12S-HPETE.pdb	  anandamide.pdb		 MRS-1477.pdb
2-APB.pdb	  arvanil.pdb			 NADA.pdb
5S-HETE.pdb	  asivatrep.pdb			 olvanil.pdb
5S-HPETE.pdb	  AZD1386.pdb			 phenylacetylrinvanil.pdb
A-1165442.pdb	  BCTC.pdb			 piperine.pdb
A425619.pdb	  camphor.pdb			 PPAHV.pdb
A778317.pdb	  capsaicin.pdb			 resiniferatoxin.pdb
agatoxin_489.pdb  capsazepine.pdb		 SB366791.pdb
allicin.pdb	  diphenylboronic_anhydride.pdb  SB452533.pdb
AMG517.pdb	  JNJ17203212.pdb		 SB705498.pdb
AMG628.pdb	  JYL1421.pdb			 zucapsaicin.pdb
AMG_9810.pdb	  mavatrep.pdb


In [16]:
def vina_dock(receptor, ligand):
    v = Vina(sf_name='vina')
    v.set_receptor(receptor) # receptor pdbqt path
    v.set_ligand_from_file(ligand) # ligand pdbqt path
    v.compute_vina_maps(center=[-23.075, -2.183, -7.899], box_size=[30, 30, 30])

    # Dock the ligand
    v.dock(exhaustiveness=32, n_poses=20)
    v.write_poses('1iep_ligand_vina_out.pdbqt', n_poses=5, overwrite=True)

receptor = 'receptor/apo_8gfa.pdbqt'
output_dir = 'IUPHAR/IUPHAR_dock/pdbqt'
ligand = 'capsaicin'
vina_dock(receptor, os.path.join(output_dir, f'{ligand}.pdbqt'))

Computing Vina grid ... done.
Performing docking (random seed: 530775752) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************


In [13]:
import os
import dock

ligand = 'capsaicin'
lig_dir = 'IUPHAR/IUPHAR_dock/'
output_dir = 'IUPHAR/IUPHAR_dock/pdbqt'
os.makedirs(output_dir, exist_ok=True)
for lig in lig_dir:
    dock.prepare_pdbqt(os.path.join(os.getcwd(), lig_dir, f'{lig}.pdb'), output_dir)

Processed /home/kjs/Downloads/AutoDockTools_py3/pycode/IUPHAR/IUPHAR_dock/capsaicin.pdb to IUPHAR/IUPHAR_dock/pdbqt/capsaicin.pdbqt
Standard Output:



In [ ]:

# Score the current pose
energy = v.score()
print('Score before minimization: %.3f (kcal/mol)' % energy[0])

# Minimized locally the current pose
energy_minimized = v.optimize()
print('Score after minimization : %.3f (kcal/mol)' % energy_minimized[0])
v.write_pose('1iep_ligand_minimized.pdbqt', overwrite=True)

# Dock the ligand
v.dock(exhaustiveness=32, n_poses=20)
v.write_poses('1iep_ligand_vina_out.pdbqt', n_poses=5, overwrite=True)